In [ ]:
import math
import numpy as np
import random
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
# Creating hrrs
def hrr(length, normalized=False):
    if normalized:
        x = np.random.uniform(-np.pi,np.pi,int((length-1)/2))
        if length % 2:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.exp(-1j*x[::-1])])))
        else:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.ones(1), np.exp(-1j*x[::-1])])))
    else:
        x = np.random.normal(0.0, 1.0/np.sqrt(length), length)
    return x

In [ ]:
# Convolve two hrrs
def convolve(x, y):
    return np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))

# Pre convolve all hrrs to save time
def preconvolve():
    preconvolved_matrix = np.zeros([possible_wm.size, size_of_maze, n])
    for x in range(len(goal_states)):
        for y in range(size_of_maze):
            preconvolved_matrix[x][y] = convolve(possible_wm[x], state_hrrs[y])
    return preconvolved_matrix

In [ ]:
# Switching between tasks
def switch_task(count):
    if count < 9:
        goal = goal_states[0]
        count += 1
    elif count == 9:
        goal = goal_states[1]
        count += 1
    elif count > 9 and count < 19:
        goal = goal_states[1]
        count += 1
    elif count == 19:
        goal = goal_states[0]
        count = 0
    return goal, count

In [ ]:
# Neural network class for wm
class NeuralNetwork:
    def __init__(self, inputs, outputs, discount, alpha):
        self.discount = discount
        self.alpha = alpha
        self.input = inputs
        self.output = outputs
        self.weights = hrr(n, True)
        self.bias = 0
        
    # To get value of each state through weights
    def feedforward(self, X):
        self.output = np.dot(self.weights, X) + self.bias
        return self.output
    
    # Backprop to update weights
    def backprop(self, state_prime_hrr, state_hrr, wm, y):
        
        # Update for reward found
        if(y == reward_good):
            self.error = (reward_bad + self.discount * self.feedforward(state_hrr)) - self.feedforward(state_prime_hrr)
            self.weights = np.add(self.weights, (self.alpha * self.error * state_prime_hrr))
            self.error = reward_good - self.feedforward(state_hrr)
            self.weights = np.add(self.weights, (self.alpha * self.error * state_hrr))
            
        # Reward not found    
        else:
            self.error = (reward_bad + self.discount * self.feedforward(state_hrr)) - self.feedforward(state_prime_hrr)
            # Switch wm if error too large
            if self.error < threshold:
                return policy_switch(wm)
            self.weights = np.add(self.weights, (self.alpha * self.error * state_prime_hrr))
        return wm

In [ ]:
# Policy for choosing left or right
def policy_mov(state, wm, rand_on):
    
    # Calculates what the left and right positions on the maze are
    if (state == 0):
        state_left = size_of_maze - 1
        state_right = state + 1
    elif (state == size_of_maze - 1):
        state_left = state - 1
        state_right = 0
    else:
        state_left = state - 1
        state_right = state + 1
    
    # Value of each move
    value_left_state_wm = nn.feedforward(preconvolved_matrix[wm][state_left])
    value_right_state_wm = nn.feedforward(preconvolved_matrix[wm][state_right])
    
    # Random move
    if((np.random.random() < e_soft) and (rand_on == 1)):
        return np.random.choice([state_left, state_right])
    
    # Finds best move
    max_value = max(value_left_state_wm, value_right_state_wm)
    
    # Returns best move
    if(max_value == value_left_state_wm):
        return state_left
    elif(max_value == value_right_state_wm):
        return state_right
    
# Switching wm depending on td error
def policy_switch(wm):
    return (wm + 1)%2

In [ ]:
# Size of hrrs
n = 1024

# Maze creation
size_of_maze = 5

# Reward for learning
reward_bad = 0
reward_good = 1

# Discounted future rewards
discount = 0.5

# Learning rate
alpha = 0.03

# Exploration on or off
exp_on = 1

# Exploration rate
e_soft = 0.01

# Treshold to switch wm
threshold = -0.20

# Number of training cycles
episodes = 5000

# Steps to try finding the goal before quiting
steps_till_quit = 100

# For switching tasks
count = -1

# Frequency to print information
print_freq = 1000

# wm is set to first
wm = 0

# Creating the neural net
nn = NeuralNetwork(n, 1, discount, alpha)

In [ ]:
# Setting goals
goal_states = random.sample(range(0, size_of_maze), 2)

# Create all state hrrs
state_hrrs = np.zeros([size_of_maze, n])
for x in range(size_of_maze):
    state_hrrs[x] = hrr(n, True)
    
# Create all wm hrrs
possible_wm = np.zeros([len(goal_states), n])
for x in range(len(goal_states)):
    possible_wm[x] = hrr(n, True)

# Reward hrr
reward_tkn = hrr(n, True)

# Pre convolve hrrs to save time
preconvolved_matrix = preconvolve()

In [ ]:
print("Goals:", goal_states[0], goal_states[1])
for x in range(episodes):
    
    # Current state
    current = random.randint(0, size_of_maze - 1)
    
    # Switching tasks
    goal_state, count = switch_task(count)

    for y in range (steps_till_quit):
        
        # Store info about previous state
        previous = current
        previous_state = preconvolved_matrix[wm][previous]
        previous_value = nn.feedforward(previous_state)
        
        # What state to move into
        move = policy_mov(current, wm, exp_on)
        
        # Make the move
        current = move
        current_state = preconvolved_matrix[wm][current]
        current_value = nn.feedforward(current_state)
              
        # Goal reached
        if (current == goal_state):
            # Get temporal difference error and update weights of neural network
            nn.backprop(previous_state, convolve(reward_tkn, current_state), wm, reward_good)
            break

        # Weight update for goal not found
        wm = nn.backprop(previous_state, current_state, wm, reward_bad)
        
    if (x+1)%print_freq == 0:
        print("Episode", x+1, "done")

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == goal_states[0]:
        value[z] = nn.feedforward(convolve(reward_tkn, preconvolved_matrix[0][z]))
    else:
        value[z] = nn.feedforward(preconvolved_matrix[0][z])
plt.plot(position, value)
print(np.amax(value))

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == goal_states[1]:
        value[z] = nn.feedforward(convolve(reward_tkn, preconvolved_matrix[1][z]))
    else:
        value[z] = nn.feedforward(preconvolved_matrix[1][z])
plt.plot(position, value)
print(np.amax(value))